In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

file_name = '/content/drive/My Drive/新生訓練/新訓HW2/ratings_small.csv'
data = pd.read_csv(file_name, encoding='utf-8', engine='python')
data.head(2)


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [ ]:
data.isna().sum()
# 無須處理空值

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [ ]:
data['rating'] = data['rating'] / data['rating'].max()
print(data['rating'].max())
data.head(2)

1.0


,userId,movieId,rating,timestamp
0,1,31,0.5,1260759144
1,1,1029,0.6,1260759179


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

target = data['rating']
del data['rating']

x_train,x_test,y_train,y_test = train_test_split(data,target,train_size=0.8,random_state = 5)

# XGBoost模型訓練

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import LeaveOneOut

# fit model no training data
model = XGBClassifier(booster='gbtree', objective ='reg:logistic')
model.fit(x_train, y_train)
y_predict = model.predict(x_test)


# LeaveOneOut
### 總共100005筆會跑很久，故選前1000筆計算

In [ ]:
from numpy import mean
from numpy import std
import numpy as np
cv = LeaveOneOut()
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, 5*y_test.to_numpy()[:1000][:,np.newaxis], 5*y_predict[:1000][:,np.newaxis], scoring='explained_variance', cv=cv, n_jobs=-1, error_score="raise")
# report performance
print(scores)
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))


[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

# Top-n Hit Rate
Assume n = 3

選擇順序
1. rating
2. timestamp(越大=>越近期的評分標準=>優先)
3. movieId

In [ ]:
y_predict = model.predict(data)

In [ ]:
# 把原本限制在0~1的值域改回data的0~5
target = 5*(target.to_numpy())
y_predict = 5*y_predict
print(target.max(), y_predict.max())

5.0 5.0


In [ ]:
# 把predict rating值加上去
data['pre_rating'] = y_predict
data.head(30)

,userId,movieId,rating,timestamp,pre_rating
0,1,31,2.5,1260759144,4.0
1,1,1029,3.0,1260759179,4.0
2,1,1061,3.0,1260759182,4.0
3,1,1129,2.0,1260759185,4.0
4,1,1172,4.0,1260759205,4.0
5,1,1263,2.0,1260759151,4.0
6,1,1287,2.0,1260759187,4.0
7,1,1293,2.0,1260759148,4.0
8,1,1339,3.5,1260759125,4.0
9,1,1343,2.0,1260759131,4.0


In [176]:
max_userid = data["userId"].max() # 確認過了，userId的編號是一個一個遞增的流水號
user_num = 0
n = 3
hit = 0
for i in range(1, max_userid+1):
  filter = (data["userId"] == i)
  target_user = data[filter].sort_values(by=['timestamp'], ascending=False)
    
  top_3_index = target_user['rating'].nlargest(n).index
  top_3_movie = []
  pre_top_3_index = target_user['pre_rating'].nlargest(n).index
  pre_top_3_movie = []
  for j in range(n):
    top_3_movie.append(target_user['movieId'][top_3_index[j]])
    pre_top_3_movie.append(target_user['movieId'][pre_top_3_index[j]])
  for j in range(n):
    if(pre_top_3_movie[j] in top_3_movie):
      hit = hit + 1
print(hit/(3*max_userid)) # Top-n Hit Rate

0.17734724292101342
